In [1]:
# Cell 1: Import libraries and fix TensorFlow installation
# Uninstall any existing TensorFlow to avoid conflicts
!pip uninstall tensorflow tf-nightly -y
# Install stable TensorFlow version
!pip install tensorflow
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# Install tensorflow-datasets if needed
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Verify TensorFlow version and imports
print("TensorFlow Version:", tf.__version__)
print("Tokenizer imported:", Tokenizer)
print("pad_sequences imported:", pad_sequences)

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_nightly 2.21.0.dev20251013
Uninstalling tf_nightly-2.21.0.dev20251013:
  Successfully uninstalled tf_nightly-2.21.0.dev20251013
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is in

In [2]:
# Cell 2: Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-10-16 22:35:02--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.07s   

2025-10-16 22:35:02 (4.68 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-10-16 22:35:02--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [3]:
# Cell 3: Data preprocessing, model building, and training
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

vocab_size = 10000
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data['message'])

train_sequences = tokenizer.texts_to_sequences(train_data['message'])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_data['message'])
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

embedding_dim = 16
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 30
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels), verbose=2)

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


131/131 - 2s - 18ms/step - accuracy: 0.8607 - loss: 0.4011 - val_accuracy: 0.8657 - val_loss: 0.3718
Epoch 2/30
131/131 - 1s - 6ms/step - accuracy: 0.8660 - loss: 0.3689 - val_accuracy: 0.8657 - val_loss: 0.3673
Epoch 3/30
131/131 - 1s - 10ms/step - accuracy: 0.8660 - loss: 0.3631 - val_accuracy: 0.8657 - val_loss: 0.3589
Epoch 4/30
131/131 - 1s - 6ms/step - accuracy: 0.8660 - loss: 0.3509 - val_accuracy: 0.8657 - val_loss: 0.3501
Epoch 5/30
131/131 - 1s - 7ms/step - accuracy: 0.8660 - loss: 0.3266 - val_accuracy: 0.8657 - val_loss: 0.3105
Epoch 6/30
131/131 - 1s - 7ms/step - accuracy: 0.8660 - loss: 0.2772 - val_accuracy: 0.8657 - val_loss: 0.2496
Epoch 7/30
131/131 - 1s - 10ms/step - accuracy: 0.8971 - loss: 0.2072 - val_accuracy: 0.9325 - val_loss: 0.1804
Epoch 8/30
131/131 - 1s - 10ms/step - accuracy: 0.9526 - loss: 0.1451 - val_accuracy: 0.9641 - val_loss: 0.1338
Epoch 9/30
131/131 - 2s - 11ms/step - accuracy: 0.9698 - loss: 0.1051 - val_accuracy: 0.9705 - val_loss: 0.1036
Epoch 1

In [4]:
# Cell 4: Function to predict messages based on model
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    prediction = model.predict(padded, verbose=0)[0][0]
    label = 'spam' if prediction >= 0.5 else 'ham'
    return [float(prediction), label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

[0.018774544820189476, 'ham']


In [5]:
# Cell 5: Test function (do not modify)
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
